# 2. Finding an upper bound on CHSH, symmetrized version

We initialize the RepLAB library.

In [1]:
run ../../../replab_init

Adding RepLAB to the path
Adding RepLAB package to the path
Loading optim package for Octave...
OpenJDK 64-Bit Server VM warning: Archived non-system classes are disabled because the java.system.class.loader property is specified (value = "org.octave.OctClassLoader"). To use archived non-system classes, this property must be not be set
Adding VPI to the path
Adding MOxUnit to the path
Adding embedded YALMIP to the path
Adding embedded SDPT3 solver to the path
Adding MOcov to the path


During the lecture, we found a form of the SDP that had only one optimization variable. We'll now see how to symmetrize the problem completely.

We first declare the one and only variable, before constructing the moment matrix.

In [20]:
y = sdpvar; % y is actually y_A0B0, but we save space by identifying y with it
C = [ 1  0  0  0  0
      0  1  0  0  0
      0  0  1  0  0
      0  0  0  1  0
      0  0  0  0  1];
A = [ 0  0  0  0  0
      0  0  0  1  1
      0  0  0  1 -1
      0  1  1  0  0
      0  1 -1  0  0];
X = C + A*y;
I_CHSH = 4*y; % it is <A0B0> + <A0B1> + <A1B0> - <A1B1>, and y_A1B1 = -y_A0B0.
optimize(X >= 0, -I_CHSH, sdpsettings('verbose', 0)); % sign change to maximize, and we don't show solver out
double(I_CHSH)

ans =  2.8284


To help with symmetrization, we'll use an equivalent form of the constraint X, and check the result.

Now, we want to find a change of basis that brings X into a block-diagonal form. This is easier to check on the basis matrices `C` and `A`, i.e. we want to find `U` such that `U*C*U'` and `U*A*U'` are block-diagonal. Let us use RepLAB for that.
First, remark that the moment matrix is invariant under the following signed permutations:

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, B_0, B_1, A_0, A_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, -A_0, -A_1, -B_0, -B_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, A_1, A_0, B_0, -B_1 \right) $

We now show how to use RepLAB to find the change of basis for the symmetry group that includes only the first symmetry.

In [4]:
g1 = [1 4 5 2 3]; % permutation of parties
g2 = [1 -2 -3 -4 -5]; % sign flip everywhere, note the signed permutation convention
g3 = [1 3 2 4 -5]; % additional symmetry

We build the symmetry group from those generators, which are signed permutations on five elements. For the permutation of parties only, the order (=size) of the group is 2. When adding the other symmetries, the order of the group should be 16.

In [5]:
nElements = 5;
G = replab.signed.Permutations(nElements).subgroup({g1 g2 g3});
G.order

ans =
    16


If necessary, we can also expand the group elements (use `G.elements.at(2)` to get the second element for example).

In [6]:
G.elements

ans =

Indexed family of 16 elements
 1 = [1, 2, 3, 4, 5]    
 2 = [1, -3, -2, -4, 5] 
 3 = [1, 2, -3, 5, 4]   
 4 = [1, 3, -2, -5, 4]  
 5 = [1, 4, 5, 2, 3]    
 6 = [1, -5, -4, -2, 3] 
 7 = [1, 4, -5, 3, 2]   
 8 = [1, 5, -4, -3, 2]  
 9 = [1, -4, 5, -3, -2] 
10 = [1, -5, 4, 3, -2]  
11 = [1, -4, -5, -2, -3]
12 = [1, 5, 4, 2, -3]   
13 = [1, -2, 3, -5, -4] 
14 = [1, -3, 2, 5, -4]  
15 = [1, -2, -3, -4, -5]
16 = [1, 3, 2, 4, -5]   



Now, the representation we need is composed of signed permutation matrices; as we are considering a group of signed permutations, this is the group *natural representation*. Representations in RepLAB are described using the images of the generators. By calling `G.sampleUniformly`, we get random elements from the group, and by calling `rep.image(g)` we get the image of a group element.

In [7]:
rep = G.naturalRep

rep =

Orthogonal representation by images
                 dimension: 5                                
                     field: 'R'                              
   frobeniusSchurIndicator: []                               
                     group: replab.signed.PermutationSubgroup
    inverseImages_internal: 1 x 3 cell                       
isDivisionAlgebraCanonical: []                               
             isIrreducible: []                               
                 isUnitary: true                             
          trivialDimension: []                               
        images_internal{1}: 5 x 5 double                     
        images_internal{2}: 5 x 5 double                     
        images_internal{3}: 5 x 5 double                     



In [8]:
g = G.sampleUniformly
full(rep.image(g)) % we use full, as signed permutation representations return sparse matrices

g =

   1  -2   3  -5  -4

ans =

   1   0   0   0   0
   0  -1   0   0   0
   0   0   1   0   0
   0   0   0   0  -1
   0   0   0  -1   0



Now, we decompose the representation into irreducible components. For that, we call `rep.decomposition`. In the answer I(m)xR(d), we express that the component has $m$ copies (multiplicity) of a `R`eal representation of dimension `d`. We then play with the indices of the component, and copy.

Hint: explore `rep.decomposition.component(i).irrep(j)` for $(i,j) = (1,1), (2,1), (3,1)$.

In [9]:
rep.decomposition

ans =

Orthogonal representation
                 dimension: 5                                             
                     field: 'R'                                           
   frobeniusSchurIndicator: []                                            
                     group: replab.signed.PermutationSubgroup             
isDivisionAlgebraCanonical: []                                            
             isIrreducible: []                                            
                 isUnitary: true                                          
                    parent: Orthogonal representation by images           
          trivialDimension: []                                            
               basis.(:,1): [1, -9.0724e-18, 0, -1.3657e-17, 1.704e-18].' 
               basis.(:,2): [0, -0.5, 0.5, 3.1845e-17, -0.70711].'        
               basis.(:,3): [0, 0.5, 0.5, 0.70711, 2.1024e-17].'          
               basis.(:,4): [0, 0.54012, 0.45637, -0.70462, -0.0592

In [10]:
rep.decomposition.component(1).irrep(1)

ans =

Orthogonal trivial irreducible real-type subrepresentation
                 dimension: 1                                            
                     field: 'R'                                          
   frobeniusSchurIndicator: 1                                            
                     group: replab.signed.PermutationSubgroup            
isDivisionAlgebraCanonical: []                                           
             isIrreducible: true                                         
                 isUnitary: true                                         
                    parent: Orthogonal representation by images          
          trivialDimension: 1                                            
               basis.(:,1): [1, -9.0724e-18, 0, -1.3657e-17, 1.704e-18].'



We now ask for the change of basis matrix, and verify that `A` and `C` block-diagonalize.

In [21]:
U = rep.decomposition.basis
U'*C*U
U'*A*U

U =

   1.00000   0.00000   0.00000   0.00000   0.00000
  -0.00000  -0.50000   0.50000   0.54012   0.45637
   0.00000   0.50000   0.50000   0.45637  -0.54012
  -0.00000   0.00000   0.70711  -0.70462   0.05922
   0.00000  -0.70711   0.00000  -0.05922  -0.70462

ans =

   1.0000e+00   3.3313e-18  -1.4193e-17   4.6221e-18  -6.1499e-18
   3.3313e-18   1.0000e+00  -4.7860e-17   1.6653e-16   1.6653e-16
  -1.4193e-17  -4.7860e-17   1.0000e+00   4.4284e-16  -9.3640e-19
   4.6221e-18   1.6653e-16   4.4284e-16   1.0000e+00  -9.0206e-17
  -6.1499e-18   1.6653e-16  -9.3640e-19  -9.0206e-17   1.0000e+00

ans =

   2.1689e-34   4.7112e-18  -2.0072e-17  -6.5366e-18   8.6972e-18
   4.7112e-18   1.4142e+00  -7.6535e-17   2.5674e-16   1.1102e-16
  -2.0072e-17  -4.6660e-17   1.4142e+00   6.5956e-16  -1.4762e-16
  -6.5366e-18   2.2898e-16   5.5687e-16  -1.4142e+00   1.8041e-16
   8.6972e-18   1.1102e-16  -1.3171e-16   1.8041e-16  -1.4142e+00



Once we have found the change of basis matrix, we transform our moment matrix `X`, and naturally it should have a block diagonal structure following the one of `C` and `A` -- for that, it may be necessary to kill small off-block coefficients.

In [22]:
X = (U'*C*U) + (U'*A*U)*y; % equivalent to the constraint above
optimize(X >= 0, -I_CHSH); % sign change to maximize
double(I_CHSH)


 num. of constraints =  1
 dim. of sdp    var  =  5,   num. of sdp  blk  =  1
*******************************************************************
   SDPT3: Infeasible path-following algorithms
*******************************************************************
 version  predcorr  gam  expon  scale_data
   HKM      1      0.000   1        0    
it pstep dstep pinfeas dinfeas  gap      prim-obj      dual-obj    cputime
-------------------------------------------------------------------
 0|0.000|0.000|8.0e-01|6.2e+00|5.0e+02| 5.000000e+01  0.000000e+00| 0:0:00| chol  1  1 
 1|1.000|1.000|6.5e-08|6.9e-02|4.3e+01| 3.984640e+01  4.400004e-01| 0:0:00| chol  1  1 
 2|0.933|1.000|1.5e-07|6.9e-03|2.9e+00| 3.700414e+00  8.665599e-01| 0:0:00| chol  1  1 
 3|1.000|0.949|3.5e-08|1.0e-03|2.8e-01| 3.024905e+00  2.752468e+00| 0:0:00| chol  1  1 
 4|0.987|0.988|1.1e-08|8.0e-05|3.4e-03| 2.830875e+00  2.827762e+00| 0:0:00| chol  1  1 
 5|0.989|0.989|3.5e-10|7.7e-06|3.8e-05| 2.828454e+00  2.828448e+00| 0:

As an exercice, recover an algebraic basis from `U`, guess the form of the semidefinite program when fully block-diagonal (i.e. for the full group of CHSH symmetries).
**Remark that now the problem can be solved by hand to recover the** $2\sqrt{2}$ **bound, as the SDP matrix is fully diagonal, and thus corresponds to linear inequalities.**